In [1]:
from experiments import BTI_DBF_U
import os
from functools import partial
from datasets import SimpleMNISTDataset, prepare_mnist_data, minmax_normalize
import torch
from unet import UNet, loss_bti_dbf_paper
from torchvision import transforms

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Prepare data if not already present
print("Preparing MNIST dataset...")
prepare_mnist_data()

# Transforms (tensor only for the DataLoader; the BTI code handles its own normalize)
transform_test = transforms.ToTensor()

# U-Net config for MNIST (1 channel)
unet_factory = partial(UNet, n_channels=1, num_classes=1, base_filter_num=32, num_blocks=2)

# Create test dataset / loader
test_dataset = SimpleMNISTDataset(
    path_to_data='./MNIST_Data/clean',
    csv_filename='clean.csv',
    data_transform=transform_test
)
dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=True, num_workers=2)
print(f"Test dataset size: {len(test_dataset)}")

# Variants
variants = [
    {
        "name": "paper-branch-1back",
        "train_fn": "branch",
        "unet_loss": loss_bti_dbf_paper,
        "tau": 0.15,
        "epochs": 30,
        "lr": 0.01,
        "p": 2,
        "visualize": True,
        "split": "final",
        "mask_granularity": "vector",
        "delta": False
    },
]

# Run BTI-DBF (U) with evaluation.
# Triggered datasets live under the usual root:
#   ./test_results/datasets/Odysseus-MNIST/Models/{model_file}_MNIST

BTI_DBF_U(
    device=device,
    experiment_name='BTIDBFU-Standard',
    num_models=5,
    model_list='./test_results/MNIST_Models_20250817_074555.csv',
    model_dir='./Odysseus-MNIST/Models',
    model_type='MNIST',
    unet_factory=unet_factory,
    dataloader=dataloader,
    variants=variants,
    mask_epochs=20,
    triggered_dataset_root="./test_results/datasets",
    unlearn_epochs=20,
    unlearn_lr=1e-3,
    feature_loss_weight=1.0,
    alt_rounds=5,
)

BTI_DBF_U(
    device=device,
    experiment_name='BTIDBFU-Fast',
    num_models=5,
    model_list='./test_results/MNIST_Models_20250817_074555.csv',
    model_dir='./Odysseus-MNIST/Models',
    model_type='MNIST',
    unet_factory=unet_factory,
    dataloader=dataloader,
    variants=variants,
    mask_epochs=20,
    triggered_dataset_root="./test_results/datasets",
    unlearn_epochs=8,
    unlearn_lr=1e-3,
    feature_loss_weight=1.0,
    alt_rounds=2,
)

Using device: cuda
Preparing MNIST dataset...


clean: 100%|█████████████████████████████| 8000/8000 [00:00<00:00, 14455.09it/s]


Saved 8000 images to ./MNIST_Data/clean
Saved CSV to ./MNIST_Data/clean/clean.csv


test: 100%|██████████████████████████████| 2000/2000 [00:00<00:00, 14554.71it/s]


Saved 2000 images to ./MNIST_Data/test
Saved CSV to ./MNIST_Data/test/test.csv
Test dataset size: 8000


...

[Mask] Epoch 1: Loss=2.3000, Benign Acc=0.1207
[Mask] Epoch 2: Loss=2.2820, Benign Acc=0.1505
[Mask] Epoch 3: Loss=2.2240, Benign Acc=0.2025
[Mask] Epoch 4: Loss=1.9774, Benign Acc=0.6693
[Mask] Epoch 5: Loss=1.0625, Benign Acc=0.9966
[Mask] Epoch 6: Loss=0.1868, Benign Acc=0.9989
[Mask] Epoch 7: Loss=0.0597, Benign Acc=0.9989
[Mask] Epoch 8: Loss=0.0358, Benign Acc=0.9992
[Mask] Epoch 9: Loss=0.0259, Benign Acc=0.9991
[Mask] Epoch 10: Loss=0.0194, Benign Acc=0.9998
[Mask] Epoch 11: Loss=0.0158, Benign Acc=0.9996
[Mask] Epoch 12: Loss=0.0132, Benign Acc=0.9998
[Mask] Epoch 13: Loss=0.0108, Benign Acc=0.9995
[Mask] Epoch 14: Loss=0.0092, Benign Acc=0.9999
[Mask] Epoch 15: Loss=0.0080, Benign Acc=0.9998
[Mask] Epoch 16: Loss=0.0073, Benign Acc=0.9998
[Mask] Epoch 17: Loss=0.0063, Benign Acc=1.0000
[Mask] Epoch 18: Loss=0.0054, Benign Acc=0.9999
[Mask] Epoch 19: Loss=0.0047, Benign Acc=0.9999
[Mask] Epoch 20: Loss=0.0041, Benign Acc=0.9996
Evaluating model: ./Odysseus-MNIST/Models/Model_7

/home/tyler/Desktop/ResearchProject/Load_Model.py:322: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(output)


Evaluating model: ./Odysseus-MNIST/Models/Model_762.pth
Dataset directory: ./test_results/datasets/Odysseus-MNIST/Models/Model_762.pth_MNIST
Dataset statistics:
  Total images: 2000
  Triggered images: 1000
  Clean images: 1000


/home/tyler/Desktop/ResearchProject/Load_Model.py:322: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(output)
BTI-DBF (U): Models: 100%|████████████████████████| 5/5 [06:52<00:00, 82.53s/it]

✅ BTI-DBF (U) results saved to MNIST_BTI_DBF_U_results.csv
